In [1]:
import pickle
from pathlib import Path
from transformers import ViTFeatureExtractor, ViTImageProcessor
from transformers import CLIPProcessor, CLIPModel
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm
import requests

/Users/samtett/Documents/Python/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configuration
INPUT_PKL = Path("/Users/samtett/Documents/Python/Project/replay_buffer_random_orientation.pkl")
VIT_OUTPUT = INPUT_PKL.parent / "replay_buffer_with_vit_embeddings.pkl"

In [3]:
class ViTMatcher:
    def __init__(self, ckpt="openai/clip-vit-base-patch32", device=None):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.proc   = ViTImageProcessor.from_pretrained(ckpt)
        self.model  = ViTModel.from_pretrained(ckpt).to(self.device).eval()

    @staticmethod
    def _l2(x):
        return x / x.norm(dim=-1, keepdim=True).clamp(min=1e-6)

    @torch.no_grad()
    def get_joint_embeddings(self, image: Image.Image, prompts: list[str]) -> np.ndarray:
        if not prompts:
            h = self.model.config.projection_dim
            return np.empty((0, h), dtype=np.float32)

        img_inp = self.proc(images=image, return_tensors="pt").to(self.device)
        img_feat = self._l2(self.model.get_image_features(**img_inp))
        txt_inp = self.proc(text=prompts, return_tensors="pt", padding=True).to(self.device)
        txt_feat = self._l2(self.model.get_text_features(**txt_inp))
        joint = self._l2(img_feat + txt_feat)
        return joint.cpu().numpy().copy()

def compute_goal_embeddings(rgb_array, tasks_dict, goal_index, color_index, embedding_size):
    pass

In [4]:
def main():
    matcher = ViTMatcher()
    matcher = ViTMatcher()
    print("🚀 Loading replay buffer...")
    with INPUT_PKL.open("rb") as f:
        buffer = pickle.load(f)

    # test_limit = 1
    print(f"Buffer size: {len(buffer.buffer)}")
    
    observations = np.array([experience[0] for experience in buffer.buffer])#[:test_limi
    actions = np.array([experience[1] for experience in buffer.buffer])#[:test_limit]
    rewards = np.array([experience[2] for experience in buffer.buffer])#[:test_limit]
    # next_observations = np.array([experience[3] for experience in buffer.buffer])#[:test_limit]
    dones = np.array([experience[4] for experience in buffer.buffer])#[:test_limit]

    rgb = np.array([obs['rgb'] for obs in observations]).transpose(0, 1, 4, 2, 3)  # (episodes, steps, channels, height, width)
    # next_rgb = np.array([obs['rgb'] for obs in next_observations]).transpose(0, 1, 4, 2, 3)  # (episodes, steps, channels, height, width)
    goal_index = np.array([obs['goal_index'] for obs in observations])  # (episodes, steps, 1)

    if rgb is None or goal_index is None:
        print("❌ Missing required data in buffer: rgb, next_rgb or goal_index.")
        return

    num_episodes, num_steps = rgb.shape[0], rgb.shape[1]
    print(f"✅ Loaded buffer with {num_episodes} episodes, {num_steps} steps each.")

    print("💬 Computing joint embeddings...")
    for ep in tqdm(range(num_episodes), desc="Embedding Episodes"):
        for st in range(num_steps):
            task_list = tasks_dict[ep][st]

            img_np = rgb[ep, st]
            if isinstance(img_np, torch.Tensor):
                img_np = img_np.cpu().numpy()
            img_np = img_np.transpose(1, 2, 0).astype(np.uint8)
            if img_np.shape[-1] == 4:
                img_np = img_np[..., :3]

            img_pil = Image.fromarray(img_np)

            if not task_list:
                # No object in frame → Add "move around" task
                prompts = ["No balls in frame, move around"]
                embs = matcher.get_joint_embeddings(img_pil, prompts)
                tasks_dict[ep][st] = [{
                    "task": prompts[0],
                    "location": None,
                    "colour": None,
                    "bbox": None,
                    "embedding": embs[0]
                }]
            else:
                prompts = [t["task"] for t in task_list]
                embs = matcher.get_joint_embeddings(img_pil, prompts)
                for t, e in zip(task_list, embs):
                    t["embedding"] = e

            if not task_list:
                # No object in frame → Add "move around" task
                prompts = ["No balls in frame, move around"]
                embs = matcher.get_joint_embeddings(img_pil, prompts)
                tasks_dict[ep][st] = [{
                    "task": prompts[0],
                    "location": None,
                    "colour": None,
                    "bbox": None,
                    "embedding": embs[0]
                }]
            else:
                prompts = [t["task"] for t in task_list]
                embs = matcher.get_joint_embeddings(img_pil, prompts)
                for t, e in zip(task_list, embs):
                    t["embedding"] = e

    # Compute final embeddings based on goal_index
    print("📦 Computing goal-based embeddings...")
    embedding_size = matcher.model.config.text_config.projection_size
    print(f"Embedding size: {embedding_size}")
    color_index = {
        "red": 0,
        "green": 1,
        "blue": 2,
        "yellow": 3,
        "magenta": 4
    }

    embeddings = compute_goal_embeddings(rgb, tasks_dict, goal_index, color_index, embedding_size)

    buffer_with_embeddings = []
    buffer_with_embeddings.append(embeddings)
    buffer_with_embeddings.append(actions)
    buffer_with_embeddings.append(rewards)
    buffer_with_embeddings.append(dones)

In [6]:
print("💾 Saving final buffer with embeddings...")
with open("vit_embeddings.pkl", "wb") as f:
    pickle.dump(VIT_OUTPUT, f, protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump(VIT_OUTPUT, f, protocol=pickle.HIGHEST_PROTOCOL)

    print(f"✅ Done. Embeddings saved to: {VIT_OUTPUT}")
    print(f"✅ Done. Embeddings saved to: {VIT_OUTPUT}")

💾 Saving final buffer with embeddings...
✅ Done. Embeddings saved to: /Users/samtett/Documents/Python/Project/replay_buffer_with_vit_embeddings.pkl
✅ Done. Embeddings saved to: /Users/samtett/Documents/Python/Project/replay_buffer_with_vit_embeddings.pkl
